In [1]:
!pip install -q protobuf==3.20.3
!pip install -q transformers --upgrade
!pip install -q accelerate --upgrade
!pip install -q peft --upgrade
!pip install -q bitsandbytes --upgrade
!pip install -q datasets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, bu

In [4]:
# ===== CELL 1: Load và xử lý dữ liệu =====
import json
import os
import random
from typing import List, Dict, Optional

def load_jsonl(path: str, fallback_label: Optional[str] = None) -> List[Dict]:
    """Load JSONL file từ Google Drive"""
    data = []
    if not path or not os.path.exists(path):
        print(f"⚠️ File không tồn tại: {path}")
        return data
    
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                continue
            answer = obj.get("answer") or fallback_label
            if answer is None:
                continue
            obj["answer"] = answer
            data.append(obj)
    
    print(f"✅ Đã load {len(data)} mẫu từ {path}")
    return data


def normalize_answer(ans: str) -> str:
    """Chuẩn hóa câu trả lời về Đúng/Sai"""
    ans = ans.strip().lower()
    if ans.startswith("đ"):
        return "Đúng"
    if ans.startswith("s"):
        return "Sai"
    return "Sai"


def build_messages(context: str, statement: str, answer: str) -> List[Dict[str, str]]:
    """Tạo format chat messages cho Qwen"""
    display_context = context.strip() or statement.strip()
    display_statement = statement.strip() or context.strip()

    user_prompt = (
        "Ngữ cảnh: {context}\n"
        "Mệnh đề: {statement}\n"
        "Hãy phân loại mệnh đề trên là 'Đúng' hoặc 'Sai'. "
        "Chỉ trả lời đúng một từ: Đúng hoặc Sai."
    ).format(context=display_context, statement=display_statement)

    return [
        {
            "role": "system",
            "content": "Bạn là chuyên gia y tế, chỉ trả lời 'Đúng' hoặc 'Sai'.",
        },
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": answer},
    ]


# ===== CẤU HÌNH ĐƯỜNG DẪN =====
TRAIN_FILE = "/kaggle/input/data-finetune/checkpoint_results.jsonl"
EXTRA_FILE = "/kaggle/input/data-finetune/checkpoint_results_sai.jsonl"

# ===== LOAD DỮ LIỆU =====
print("📊 Đang load dữ liệu...")
train_data = load_jsonl(TRAIN_FILE)
extra = load_jsonl(EXTRA_FILE) if EXTRA_FILE else []
all_data = train_data + extra

# Shuffle dữ liệu
SEED = 42
random.seed(SEED)
random.shuffle(all_data)

if len(all_data) == 0:
    raise ValueError("❌ Không có dữ liệu để huấn luyện. Kiểm tra đường dẫn file!")

# Split train/eval (95%/5%)
split_idx = int(0.95 * len(all_data))
train_examples = all_data[:split_idx]
eval_examples = all_data[split_idx:]

print(f"\n✅ Tổng số mẫu: {len(all_data)}")
print(f"   - Train: {len(train_examples)}")
print(f"   - Eval: {len(eval_examples)}")
print(f"\n📦 Dữ liệu đã sẵn sàng cho training!")

📊 Đang load dữ liệu...
✅ Đã load 20000 mẫu từ /kaggle/input/data-finetune/checkpoint_results.jsonl
✅ Đã load 20000 mẫu từ /kaggle/input/data-finetune/checkpoint_results_sai.jsonl

✅ Tổng số mẫu: 40000
   - Train: 38000
   - Eval: 2000

📦 Dữ liệu đã sẵn sàng cho training!


In [12]:
# ===== CELL: TRAINING (CUSTOM MODE - BATCH 6) =====
import os
import gc
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    Trainer, TrainingArguments, DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# --- 1. DỌN DẸP & CẤU HÌNH ---
gc.collect()
torch.cuda.empty_cache()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# --- 2. HÀM XỬ LÝ (Giữ nguyên) ---
def normalize_answer(ans):
    if not isinstance(ans, str): return "Sai"
    ans = ans.strip().lower()
    if ans.startswith("đ"): return "Đúng"
    if ans.startswith("s"): return "Sai"
    return "Sai"

def build_messages(context, statement, answer):
    display_context = context.strip() if context else (statement.strip() if statement else "")
    display_statement = statement.strip() if statement else display_context
    user_prompt = (
        f"Ngữ cảnh: {display_context}\n"
        f"Mệnh đề: {display_statement}\n"
        "Hãy phân loại mệnh đề trên là 'Đúng' hoặc 'Sai'. "
        "Chỉ trả lời đúng một từ: Đúng hoặc Sai."
    )
    return [
        {"role": "system", "content": "Bạn là chuyên gia y tế, chỉ trả lời 'Đúng' hoặc 'Sai'."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": answer},
    ]

# --- 3. DATASET & MODEL ---
class QwenLazyDataset(Dataset):
    def __init__(self, examples, tokenizer, max_len):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.examples)
    def __getitem__(self, idx):
        example = self.examples[idx]
        single_text = example.get("text") or example.get("sentence") or example.get("content") or ""
        context = example.get("context", "") or single_text
        statement = example.get("statement", "") or single_text
        answer = normalize_answer(example.get("answer", "Sai"))
        
        messages = build_messages(context, statement, answer)
        prompt_text = self.tokenizer.apply_chat_template(messages[:-1], tokenize=False, add_generation_prompt=True)
        full_text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        
        prompt_ids = self.tokenizer(prompt_text, truncation=True, max_length=self.max_len)["input_ids"]
        enc = self.tokenizer(full_text, truncation=True, max_length=self.max_len, padding=False, return_tensors="pt")
        
        input_ids = enc["input_ids"][0]
        labels = input_ids.clone()
        labels[:len(prompt_ids)] = -100
        return {"input_ids": input_ids, "attention_mask": enc["attention_mask"][0], "labels": labels}

OUTPUT_DIR = "/kaggle/working/Checkpoint/qwen3_slm_batch6"
MODEL_ID = "Qwen/Qwen3-0.6B"

print(f"🚀 Đang tải model {MODEL_ID}...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, quantization_config=bnb_config, torch_dtype=torch.bfloat16,
    device_map="auto", trust_remote_code=True
)
model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_kbit_training(model)

# --- BẮT BUỘC: Checkpointing ---
model.gradient_checkpointing_enable()

lora_config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

# --- 4. TRAINER CONFIG ---
print("📦 Đang chuẩn bị dataset...")
train_dataset = QwenLazyDataset(train_examples, tokenizer, max_len=512)
eval_dataset = QwenLazyDataset(eval_examples, tokenizer, max_len=512)
data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    # --- CẤU HÌNH BATCH 6 ---
    per_device_train_batch_size=6,    # Tăng lên 6
    gradient_accumulation_steps=5,    # 6 * 5 = 30 (Gần tương đương batch size 32 chuẩn)
    gradient_checkpointing=True,      # GIỮ NGUYÊN để không bị lỗi bộ nhớ
    dataloader_num_workers=2,         
    # ------------------------
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=200,
    eval_strategy="steps",
    eval_steps=200,
    save_total_limit=2,
    bf16=True,
    fp16=False,
    optim="paged_adamw_8bit",
    report_to="none",
    ddp_find_unused_parameters=False,
    group_by_length=True,
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_args,
    data_collator=data_collator,
)

print("\n🏎️ Bắt đầu training (Batch 6)...")
model.config.use_cache = False
trainer.train()

model.config.use_cache = True
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Hoàn tất!")

🚀 Đang tải model Qwen/Qwen3-0.6B...
📦 Đang chuẩn bị dataset...

🏎️ Bắt đầu training (Batch 6)...


Step,Training Loss,Validation Loss
200,0.008900,0.005638
400,0.008800,0.003404
600,0.004600,0.010995
800,0.004600,0.002575
1000,0.003400,0.004094
1200,0.002600,0.002831
1400,0.003200,0.003416
1600,0.002700,0.002759
1800,0.002600,0.003132
2000,0.000700,0.003606


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

✅ Hoàn tất!


In [ ]:
# ==========================================
# 1. CẤU HÌNH & LOAD MODEL (ĐÃ SỬA LỖI)
# ==========================================
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Đường dẫn (Giữ nguyên của bạn)
ADAPTER_PATH = "/kaggle/working/Checkpoint/qwen3_slm_batch6" 
BASE_MODEL_ID = "Qwen/Qwen3-0.6B"
TEST_FILE_PATH = "/kaggle/input/data-test/data_test_normalize.csv" 

print("🚀 Đang load model để kiểm tra...")

# --- SỬA ĐỔI 1: Load Tokenizer từ ADAPTER_PATH (thay vì BASE_MODEL_ID) ---
# Lý do: Để lấy đúng cái tokenizer đã dùng lúc train
try:
    tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH, trust_remote_code=True)
except:
    # Fallback nếu không tìm thấy trong adapter path
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)

if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token

# Load model gốc
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.bfloat16, # Hoặc float16 tùy GPU
    device_map="auto",
    trust_remote_code=True
)

# --- SỬA ĐỔI 2: RESIZE LẠI MODEL GỐC (QUAN TRỌNG NHẤT) ---
# Ép kích thước model gốc phải bằng đúng kích thước lúc train (151669)
# Nếu tokenizer load đúng, len(tokenizer) sẽ là 151669
base_model.resize_token_embeddings(len(tokenizer))

# Gắn Adapter (LoRA) vào
# Bây giờ kích thước đã khớp, lệnh này sẽ chạy ngon lành
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

print("✅ Model đã sẵn sàng!")

# ==========================================
# 2. HÀM CHUẨN HÓA & DỰ ĐOÁN
# ==========================================
def normalize_ground_truth(value):
    """
    Chuẩn hóa cột True/False trong file CSV về 'Đúng'/'Sai'
    để so sánh với model.
    """
    s = str(value).strip().lower()
    if s in ['true', '1', 't', 'yes', 'đúng']:
        return 'Đúng'
    if s in ['false', '0', 'f', 'no', 'sai']:
        return 'Sai'
    return 'Không xác định'

def predict_row(row):
    """Hàm dự đoán cho từng dòng"""
    # Lấy dữ liệu từ các cột (Sửa tên cột nếu file CSV của bạn khác)
    context = str(row.get('context', '')) 
    statement = str(row.get('statement', ''))
    
    # Format giống hệt lúc train
    display_context = context if context else statement
    display_statement = statement if statement else display_context
    
    prompt = f"""Ngữ cảnh: {display_context}
Mệnh đề: {display_statement}
Hãy phân loại mệnh đề trên là 'Đúng' hoặc 'Sai'. Chỉ trả lời đúng một từ: Đúng hoặc Sai."""

    messages = [
        {"role": "system", "content": "Bạn là chuyên gia y tế, chỉ trả lời 'Đúng' hoặc 'Sai'."},
        {"role": "user", "content": prompt}
    ]
    
    # Tokenize và chạy model
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=5, # Chỉ cần ngắn thôi
            temperature=0.01, # Gần 0 để kết quả nhất quán
            pad_token_id=tokenizer.pad_token_id
        )
        
    # Giải mã kết quả
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Lấy phần model trả lời (sau chữ assistant)
    answer = result.split("assistant")[-1].strip()
    
    # Chuẩn hóa đầu ra của model (đôi khi nó trả lời thêm dấu chấm)
    if "đúng" in answer.lower(): return "Đúng"
    if "sai" in answer.lower(): return "Sai"
    return "Sai" # Mặc định nếu model nói linh tinh

# ==========================================
# 3. CHẠY TRÊN FILE CSV
# ==========================================
# Load file CSV
try:
    df = pd.read_csv(TEST_FILE_PATH)
    print(f"📄 Đã load file CSV: {len(df)} dòng")
except Exception as e:
    print(f"❌ Lỗi load file CSV: {e}")
    # Tạo data giả để test code nếu chưa có file
    df = pd.DataFrame([
        {"context": "Bệnh nhân sốt cao 39 độ.", "statement": "Bệnh nhân bị hạ thân nhiệt.", "label": False},
        {"context": "Viêm gan B lây qua đường máu.", "statement": "Viêm gan B là bệnh truyền nhiễm.", "label": True}
    ])

# ⚠️ QUAN TRỌNG: Chuẩn hóa cột Label (Kết quả đúng) trong file CSV
# Bạn hãy thay 'label' bằng tên cột chứa True/False trong file của bạn (ví dụ: 'answer', 'Ground Truth'...)
COL_LABEL_NAME = 'answer' # <--- SỬA TÊN CỘT LABEL CỦA BẠN Ở ĐÂY

if COL_LABEL_NAME in df.columns:
    df['Chuan_Hoa'] = df[COL_LABEL_NAME].apply(normalize_ground_truth)
else:
    print(f"⚠️ Không tìm thấy cột '{COL_LABEL_NAME}'. Code sẽ chỉ chạy dự đoán.")

# Chạy dự đoán (Dùng tqdm để hiện thanh loading)
tqdm.pandas()
print("⏳ Đang chạy dự đoán...")
df['Model_Du_Doan'] = df.progress_apply(predict_row, axis=1)

# Tính độ chính xác (nếu có cột label)
if COL_LABEL_NAME in df.columns:
    df['Ket_Qua'] = df['Model_Du_Doan'] == df['Chuan_Hoa']
    acc = df['Ket_Qua'].mean()
    print(f"\n🎯 ĐỘ CHÍNH XÁC: {acc:.2%}")

# Lưu kết quả ra file mới
OUTPUT_CSV = "ket_qua_kiem_tra.csv"
df.to_csv(OUTPUT_CSV, index=False)
print(f"💾 Đã lưu kết quả tại: {OUTPUT_CSV}")

# Hiển thị 5 dòng đầu
print(df[['Model_Du_Doan', 'Chuan_Hoa']].head())

🚀 Đang load model để kiểm tra...
✅ Model đã sẵn sàng!
📄 Đã load file CSV: 1246 dòng
⏳ Đang chạy dự đoán...


 53%|█████▎    | 664/1246 [04:33<03:56,  2.47it/s]